# Create training sub set
Create a random subset of training and val data either on a balanced set of chexpert diseases or on just one disease.

In [13]:
#hermione created: 14/07/2023
import pandas as pd
import numpy as np

train = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train.csv")
chexpert = pd.read_csv("/home/hermione/Documents/data/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv")
val = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid.csv")
train_chex = pd.merge(train, chexpert, on=['subject_id', 'study_id'])
val_chex = pd.merge(val, chexpert, on=['subject_id', 'study_id'])

#summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val"])
summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "chexpert_frac","train","train_frac", "val", "val_frac"])
diseases = chexpert.columns[2:]
chex_dis = []
train_dis = []
val_dis = []

for col in diseases:
    #print(col)
    def dis_sum(data):
        return np.sum(data[data[col]==1][col].astype(int))
    def dis_frac(data):
        subset = data.iloc[2:]
        denom = sum([(subset[subset[d] == 1][d].astype(int)).sum() for d in diseases])
        frac = dis_sum(data)/denom
        return frac
    #summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex)]
    summary[col] = [dis_sum(chexpert), dis_frac(chexpert), dis_sum(train_chex), dis_frac(train_chex), dis_sum(val_chex), dis_frac(val_chex)]
    #print(np.sum(chexpert[chexpert[col]==1][col].astype(int)))
    #print(np.sum(train_chex[train_chex[col]==1][col].astype(int)))
    #print(np.sum(val_chex[val_chex[col]==1][col].astype(int)))

print(len(chexpert), len(train), len(val))
summary #potentially make percentage


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [3]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
train_data = train_chex[train_chex.index.isin(train_data_index)| (train_chex["Pleural Effusion"] == 1)]

val_data = val_chex[val_chex.index.isin(val_data_index) | (val_chex["Pleural Effusion"] == 1)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 10433
10433 1510


In [4]:
train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_subset.csv",index=False)
val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_subset.csv", index=False)

In [6]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(n=10000, random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(n=1500, random_state=1).index

train_data = train_chex[~train_chex.index.isin(train_data_index)]
val_data = val_chex[~val_chex.index.isin(val_data_index)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

#train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_nf_eq.csv",index=False)
#val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_nf_eq.csv", index=False)

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 23510
23510 3394


In [7]:
#definitly not overlapping
print(train_final[train_final["mimic_image_file_path"].isin(val_final["mimic_image_file_path"])])

Empty DataFrame
Columns: [subject_id, study_id, image_id, mimic_image_file_path, reference_report]
Index: []
